<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os
import warnings
warnings.filterwarnings('ignore')

In [3]:
#DATAFILE_NAME = "reduced_dim_space_ddG.csv"
DATASET = "REDUCED"
if DATASET == "REDUCED":
  DATAFILE_NAME = "reduced_dim_space_ddG.csv"
  X_COLS = ['x', 'y', 'z']
else:
  DATAFILE_NAME = "large_cat_desc_col_names.csv"
  X_COLS = []
if os.path.exists('/content/drive'):
  df = pd.read_csv('/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/data/' + DATAFILE_NAME)
else:
  df = pd.read_csv('../data/' + DATAFILE_NAME)

df.columns

Index(['Catalyst', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '3965', '3966', '3967', '3968', '3969', '3970', '3971', '3972', '3973',
       'ddG'],
      dtype='object', length=3975)

In [4]:
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)
# start with a few randomly selected catalyst
START_SAMPLE_SIZE = 3
random_catalyst_df = df.sample(START_SAMPLE_SIZE)

In [5]:
random_catalyst_df

,1,2,3,4,5,6,7,8,9,10,...,3965,3966,3967,3968,3969,3970,3971,3972,3973,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
19_9_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898,0.276786
22_3_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194
17_1_6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.436071,-0.539398,-0.951100,1.110344,1.555441,1.638297,1.598277,1.521070,1.614430,0.751276


In [6]:
df

,1,2,3,4,5,6,7,8,9,10,...,3965,3966,3967,3968,3969,3970,3971,3972,3973,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
1_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194
1_1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.880495,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684
1_1_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898,0.276786
1_1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194
1_1_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633,1.463011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_9_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.880495,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684
9_9_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898,0.276786
9_9_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194


In [7]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts):
    X = df_catalysts[['x','y','z']]
    y = df_catalysts['ddG (% ee)']
    model = PLSRegression(n_components=2)
    # fitting the model
    model.fit(X, y)
    return model

In [8]:
def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=100, cd_iterations=10, step_size=0.01):
    if pls_model == None:
        print("Creating a new model. # data items: " + str(len(catalysts)))
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG (% ee)']
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
    return optimized_catalysts

new_optimized_catalysts = optimize_catalysts(random_catalyst_df[])

KeyError: "None of [Index(['x', 'y', 'z'], dtype='object')] are in the [columns]"

In [ ]:
random_catalyst_df

,x,y,z,ddG (% ee)
Catalyst,,,,
10_1_2,41.007284,-0.974831,-87.973747,0.638112
12_7_1,-43.760461,-27.829366,23.015997,0.446440
28_3_2,2.142702,22.092128,-44.495000,0.003554


In [ ]:
new_optimized_catalysts

array([[ 51.0072841 ,   1.02516941, -83.59374707],
       [-33.76046092, -25.82936563,  27.39599713],
       [ 12.14270188,  24.09212846, -40.11499953]])

In [ ]:
# Perform k-nearest neighbors analysis
num_neighbors = 1  # Number of neighbors to consider
knn = NearestNeighbors(n_neighbors=num_neighbors)
knn.fit(df[['x', 'y', 'z']])  # Using all the catalysts

NearestNeighbors(n_neighbors=1)

In [ ]:
original_ee = df['ddG (% ee)']
def get_neighbors(knn, for_points):
    knn_indices = []
    for i in range(len(for_points)):
        distances, indices = knn.kneighbors([for_points[i]])
        knn_indices.append(indices[0][0])
    return knn_indices

knn_indices = get_neighbors(knn, new_optimized_catalysts)
knn_indices

[22, 55, 211]

In [ ]:
found_neighbors = df.iloc[knn_indices]

In [ ]:
random_catalyst_df = pd.concat([random_catalyst_df, found_neighbors], ignore_index=False, sort=True)

In [ ]:
random_catalyst_df

,ddG (% ee),x,y,z
Catalyst,,,,
10_1_2,0.638112,41.007284,-0.974831,-87.973747
12_7_1,0.446440,-43.760461,-27.829366,23.015997
28_3_2,0.003554,2.142702,22.092128,-44.495000
10_3_4,0.061652,60.067650,1.412118,-82.584672
12_7_2,1.241325,-43.602601,-27.703566,23.293155
26_3_2,0.001185,9.529914,18.422417,-31.093718


In [ ]:
random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [ ]:
random_catalyst_df

,ddG (% ee),x,y,z
Catalyst,,,,
10_1_2,0.638112,41.007284,-0.974831,-87.973747
12_7_1,0.446440,-43.760461,-27.829366,23.015997
28_3_2,0.003554,2.142702,22.092128,-44.495000
10_3_4,0.061652,60.067650,1.412118,-82.584672
12_7_2,1.241325,-43.602601,-27.703566,23.293155
26_3_2,0.001185,9.529914,18.422417,-31.093718


In [ ]:
def optimize_find_catalysts(catalysts_X):
    new_optimized_catalysts = optimize_catalysts(catalysts_X)
    knn_indices = get_neighbors(knn, new_optimized_catalysts)
    found_neighbors = df.iloc[knn_indices]
    return new_optimized_catalysts, found_neighbors

In [ ]:
# We will now do all this in a single loop
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
prev_loop_len = len(random_catalyst_df)
while(len(random_catalyst_df) < len(df)):
    new_optimized_catalysts, found_neighbors = optimize_find_catalysts(random_catalyst_df[['x', 'y', 'z']])
    random_catalyst_df = pd.concat([random_catalyst_df, found_neighbors], ignore_index=False, sort=True)
    random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]
    if(len(random_catalyst_df) == prev_loop_len):
        print("The processing has blocked and no new catalysts are being added. Exiting.")
        break
    prev_loop_len = len(random_catalyst_df)
    print(prev_loop_len)

Creating a new model. # data items: 3
6
Creating a new model. # data items: 6
8
Creating a new model. # data items: 8
9
Creating a new model. # data items: 9
The processing has blocked and no new catalysts are being added. Exiting.


In [ ]:
random_catalyst_df

,ddG (% ee),x,y,z
Catalyst,,,,
3_3_4,0.264932,26.224531,7.308104,-19.972957
22_3_4,0.054528,2.863381,11.112635,-41.215495
1_3_4,0.222921,23.530906,-7.777752,-30.037466
3_3_1,0.221694,26.302667,7.368519,-19.941451
26_3_2,0.001185,9.529914,18.422417,-31.093718
6_3_1,0.210674,29.790008,-5.312876,-19.886050
24_3_4,0.007107,9.935389,29.086411,-27.455809
2_11_1,0.732504,29.115667,1.301971,-3.767900
17_3_4,0.003554,12.312513,41.899813,-26.265284
